In [1]:
import pyactr as actr

robot_mind   = actr.ACTRModel()


In [2]:
dm_module=robot_mind.decmem 
goal_buffer=robot_mind.goal
rtrv_buffer=robot_mind.retrieval


# Assumption - Robot knows the world and it is stored in declarative memory
- We consider world to be 3 x 15 grid,
- Robot will start moving in the second column, whenever there is an obstacle the robot will push the obstacle to either side.
- if obstacle slide is not possible, it will pick up and place it in behind cell.
- lets call only second column of the world as cells. 
## chunk definitions



In [3]:
actr.chunktype("world_cell", "name, left, right, block, next")# to be strored in declarative memory
actr.chunktype("goal_chunk", "start, end, current_pos, nxt_pos,status , state")
actr.chunktype("intrnl_rep","object, left, right")

## make world chunks

In [4]:
cells=[]
cells.append ( actr.makechunk(nameofchunk='w1',typename='world_cell',
                       name=1,block=False,left=False,right=False,next=2) )
cells.append ( actr.makechunk(nameofchunk='w2',typename='world_cell',
                       name=2,block=True,left=True,right=False,next=3) )
cells.append ( actr.makechunk(nameofchunk='w3',typename='world_cell',
                       name=3,block=False,left=True,right=True,next=4) )
cells.append ( actr.makechunk(nameofchunk='w4',typename='world_cell',
                       name=4,block=True,left=False,right=False,next=5) )
cells.append ( actr.makechunk(nameofchunk='w5',typename='world_cell',
                       name=5,block=True,left=True,right=True,next=6) )
cells.append (actr.makechunk(nameofchunk='w6',typename='world_cell',
                       name=6,block=False,left=True,right=False,next=7) )
cells.append (actr.makechunk(nameofchunk='w7',typename='world_cell',
                       name=7,block=True,left=False,right=True,next=8) )
cells.append ( actr.makechunk(nameofchunk='w8',typename='world_cell',
                       name=8,block=True,left=True,right=True,next=9) )
cells.append ( actr.makechunk(nameofchunk='w9',typename='world_cell',
                       name=9,block=True,left=True,right=True,next=10) )
cells.append ( actr.makechunk(nameofchunk='w10',typename='world_cell',
                       name=10,block=False,left=False,right=False,next=None) )

In [5]:
for i in cells:
    dm_module.add(i)



In [6]:
dm_module

{world_cell(block= False, left= False, name= 1, next= 2, right= False): array([0.]), world_cell(block= True, left= True, name= 2, next= 3, right= False): array([0.]), world_cell(block= False, left= True, name= 3, next= 4, right= True): array([0.]), world_cell(block= True, left= False, name= 4, next= 5, right= False): array([0.]), world_cell(block= True, left= True, name= 5, next= 6, right= True): array([0.]), world_cell(block= False, left= True, name= 6, next= 7, right= False): array([0.]), world_cell(block= True, left= False, name= 7, next= 8, right= True): array([0.]), world_cell(block= True, left= True, name= 8, next= 9, right= True): array([0.]), world_cell(block= True, left= True, name= 9, next= 10, right= True): array([0.]), world_cell(block= False, left= False, name= 10, next= None, right= False): array([0.])}

# creating goal buffer

In [7]:
#creating goal buffer
goal_buffer.add(actr.chunkstring(string="""
    isa     goal_chunk
    start   1
    end     10
    current_pos 1
    nxt_pos     2
    state INITIAL
    status FREE

"""))

In [8]:
goal_buffer

{goal_chunk(current_pos= 1, end= 10, nxt_pos= 2, start= 1, state= INITIAL, status= FREE)}

# production rules follow; using productionstring, 

In [22]:
### Initialise
robot_mind.productionstring(name="Start", string="""
    =g>
    isa     goal_chunk
    state   INITIAL
    start   =x
    current_pos =x
    nxt_pos =y
    status  FREE
    ==>
    =g>
    isa     goal_chunk
    state   SHOW                        
    +retrieval>
    isa world_cell
    name   =y
   
""")

####stop decision
robot_mind.productionstring(name="stop", string="""
    =g>
    isa     goal_chunk
    current_pos   =x
    end     =x
    status FREE
    ==>
    !g>
    show current_pos
    show done
    ~g>
""")
###show current position
robot_mind.productionstring(name='show',string="""
    =g>
    isa goal_chunk
    state SHOW
    ==>
    !g>
    show current_pos
    =g>
    isa goal_chunk
    state MAKE_DECISION
                            
                            """)

###move to cell, this operation directly fetches next cell data ater moving
robot_mind.productionstring(name="move", string="""
    =g>
    isa     goal_chunk
    state   MOVE    
    current_pos   =x
    end ~=x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    ==>
    
    =g>
    isa     goal_chunk
    current_pos   =y
    nxt_pos       =z
    state SHOW
    
    +retrieval>
    isa world_cell
    name   =z
""")

### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_no_obstacle", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status FREE
    current_pos   =x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    block False 
                    
    ==>
    =g>
    isa     goal_chunk
    state MOVE
""")

### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_obstacle_lf_rf", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status FREE
    current_pos   =x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    block True
    left  False
    right False

                    
    ==>
    !g>
    show 'push block left @' 
    show nxt_pos 
    =g>
    isa     goal_chunk
    state MOVE
""")

### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_obstacle_l_or_rf", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status FREE
    current_pos   =x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    block True
    left  =flag
    right ~=flag

                    
    ==>
    !g>
    show 'push block to free side @' 
    show nxt_pos 
    =g>
    isa     goal_chunk
    state MOVE
""")
### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_pickup", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status FREE
    current_pos   =x
    nxt_pos       =y
    =retrieval>
    isa world_cell
    name =y
    next =z
    block True
    left  True
    right True

                    
    ==>
    !g>
    show 'pick obstacle @' 
    show nxt_pos 
    =g>
    isa     goal_chunk
    state MOVE
    status LOAD
    
""")


### goal STATE is MAKE DECISOIN  , ask robot to whether to move forwards or slide othe obstacle
robot_mind.productionstring(name="MAKE_DECISION_release", string="""
    =g>
    isa     goal_chunk
    state   MAKE_DECISION
    status  LOAD
    current_pos  =x
    nxt_pos       =y
                        
    ==>
    !g>
    show 'Turn around @' 
    show current_pos
    show 'release load, and Turn around @'
    show current_pos
    =g>
    isa     goal_chunk
    status FREE
""")



{'=g': goal_chunk(current_pos= =x, end= , nxt_pos= =y, start= , state= MAKE_DECISION, status= LOAD)}
==>
{'!g': ParseResults([ParseResults(['show', "'Turn around @'"], {}), ParseResults(['show', 'current_pos'], {}), ParseResults(['show', "'release load, and Turn around @'"], {}), ParseResults(['show', 'current_pos'], {})], {}), '=g': goal_chunk(current_pos= , end= , nxt_pos= , start= , state= , status= FREE)}

In [23]:
#creating goal buffer
goal_buffer.add(actr.chunkstring(string="""
    isa     goal_chunk
    start   1
    end     10
    current_pos 1
    nxt_pos     2
    state INITIAL
    status FREE

"""))
robot_sim=robot_mind.simulation(trace=False)
robot_sim.run(max_time=2)

current_pos 1
'push block to free side @'
nxt_pos 2
current_pos 2
current_pos 3
'push block left @'
nxt_pos 4
current_pos 4
'pick obstacle @'
nxt_pos 5
current_pos 5
'Turn around @'
current_pos 5
'release load, and Turn around @'
current_pos 5
current_pos 6
'push block to free side @'
nxt_pos 7
current_pos 7
'pick obstacle @'
nxt_pos 8
current_pos 8
'Turn around @'
current_pos 8
'release load, and Turn around @'
current_pos 8
'pick obstacle @'
nxt_pos 9
current_pos 9
'Turn around @'
current_pos 9
'release load, and Turn around @'
current_pos 9
current_pos 10
done


In [20]:
goal_buffer

{goal_chunk(current_pos= 10, end= 10, nxt_pos= None, start= 1, state= SHOW, status= FREE)}